In [14]:
import os
import pathlib

import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

ROOT = pathlib.Path("D:\\ML_Dataset")
PATH = pathlib.Path(ROOT / "bite_size" / "training")
TEST_PATH = pathlib.Path("D:\\ML_Dataset\\bite_size\\test")
SAMPLE_SIZE = 50

In [15]:
def preprocess_image(image_path):
    # Load image with target size matching training input
    image = load_img(image_path, target_size=(150, 150))

    # Convert image to array
    image_array = img_to_array(image)

    # Expand dimensions to fit model input shape (batch size, height, width, channels)
    image_array = np.expand_dims(image_array, axis=0)

    # Normalize the image array as done during model training
    image_array /= 255.0

    return image_array

In [16]:
def predict_image_class(image_path):
    processed_image = preprocess_image(image_path)

    # Predict the class probabilities for the image
    prediction = model.predict(processed_image)

    # Get the class with the highest probability
    predicted_class = prediction.argmax(axis=-1)  # Retrieves the index of the highest value in the array

    return predicted_class

In [17]:
for i, path in enumerate(pathlib.Path(PATH).iterdir()):
    if i == SAMPLE_SIZE:
        break
    print(path.name)

copepods
diatoms


In [27]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(PATH,
                                                    target_size=(150, 150),
                                                    batch_size=SAMPLE_SIZE,
                                                    class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = validation_datagen.flow_from_directory(PATH,
                                                              target_size=(150, 150),
                                                              batch_size=SAMPLE_SIZE,
                                                              class_mode='binary')

Found 59 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


In [28]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [30]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16725138407659446806
xla_global_id: -1
]


In [32]:
tf.config.list_physical_devices('GPU')

[]

In [33]:
tf.test.is_built_with_cuda()

False

In [34]:
with tf.device('/GPU:0'):
    model.fit(train_generator,
              steps_per_epoch=200,
              epochs=10,
              validation_data=validation_generator,
              validation_steps=len(validation_generator))

Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6948 - loss: 0.8600 - val_accuracy: 0.7966 - val_loss: 0.5030
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6948 - loss: 0.6006 - val_accuracy: 0.7288 - val_loss: 0.3802
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6953 - loss: 0.4882 - val_accuracy: 0.6949 - val_loss: 0.4531
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - accuracy: 0.6951 - loss: 0.4318 - val_accuracy: 0.9153 - val_loss: 0.2729
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8803 - loss: 0.3578 - val_accuracy: 0.9831 - val_loss: 0.1873
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8814 - loss: 0.3158 - val_accuracy: 0.9492 - val_loss: 0.1660
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8814 - loss: 0.3073 - val_accuracy: 0.9831 - val_loss: 0.1206
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - accuracy: 0.9830 - loss: 0.1413 - val_accuracy

In [35]:
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f"Test accuracy: {test_accuracy:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9461 - loss: 0.1534
Test accuracy: 0.95


In [36]:
for pic in TEST_PATH.iterdir():
    print(f"{pic.name}: {predict_image_class(pic)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
fs446_eggs_roi0.2922506200.tif.png: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
fs446_eggs_roi1.3339894300.tif.png: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
fs446_eggs_roi3.2768170400.tif.png: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
fs446_eggs_roi3.3149332100.tif.png: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
fs446_eggs_roi3.3289047500.tif.png: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
fs534_copepods_roi1.4031192200.tif.png: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
fs534_copepods_roi1.4048441100.tif.png: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
fs534_copepods_roi1.4133164900.tif.png: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
fs534_copepods_roi1.4183715200.tif.png: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
training_diatoms_roi0.4070788301.tif.png: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
training_diatoms_roi0.4073816201.tif.png: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
training_diatoms_roi2.4413740400.tif.png: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20